todo  maybe we can just simply add more noise add more exploration?  

?? puct_base?  puct_init?  **softmax_temperature?**
- actor_mcts_puct_base : defualt 19652,  if it is smaller number (ex 5000) I guess we'll have more explore
- actor_mcts_puct_init??
- actor_select_action_softmax_temperature= : default=1, if we set it to 2 (higher temperature) it can explore more  
- actor_gumbel_sigma_scale_c??
- actor_mcts_value_rescale(dont use this)

I think dont spend too many time on the shift and power, nn is weird , just read some paper  
also  we need to set the seed for the neural network  


- note  there is a problem:-  even if we use random step ,  it is still possible that the agent just first use bad action and then clear the row, how to make sure we go to next row   
-  todo  maybe just random step the whole 2nd row  

todo  fp16, or fp8 in inference (self play mode)  

In [ ]:
python z_read_sgf_max_reward.py
#* max reward doesn't means min_comp
python z_read_sgf_min_comp.py --iter 1 --layer 1 --git 19e696
python z_read_sgf_min_comp.py --iter 1 --layer 3 --git 19e696
python z_read_sgf_min_comp.py --iter 1 --git ed1f10  #small computer  
python test.py  #to debug jit error  #todo try to modify torso

file to exclude  
amazons.*, atari.*, breakthrough.*,clobber.*,conhex.*,go.*,connect6.*,hex.*,gomoku.*,othello.*,rubiks.*,santorini.*,puzzle2048.*,surakarta.*,havannah.*,dotsandboxes.*,linesofaction.*,kernsearch.*,tictactoe.*,kernsearchtt.*

In [ ]:
python run.py --mode "podman"      
#*-------use config file-------- 
tools/quick-run.sh train kernsearch kernsearch_gaz.cfg 100  #* train for 100 iters 
#* ----- 4x4 board-----
tools/quick-run.sh train kernsearch gaz 50 -conf_str \
  env_board_size=4:actor_num_simulation=16:kern_search_game_len=20:network_type=conv:rand_init_step=0 -gen kernsearch_gaz.cfg
#* ----- 8x8 board-----
tools/quick-run.sh train kernsearch gaz 50 -conf_str \
  env_board_size=8:actor_num_simulation=16:kern_search_game_len=64:network_type=conv:rand_init_step=3 -gen kernsearch_gaz.cfg
#* ----rand_init_step 0 just wont work----  
tools/quick-run.sh train kernsearch gaz 50 -conf_str \
  env_board_size=8:actor_num_simulation=16:kern_search_game_len=64:network_type=conv:rand_init_step=0 -gen kernsearch_gaz.cfg

#----  actor_select_action_softmax_temperature is  useless
tools/quick-run.sh train kernsearch gaz 50 -conf_str \
  env_board_size=8:actor_num_simulation=16:kern_search_game_len=64:network_type=conv:rand_init_step=0:\
actor_mcts_puct_base=5000: -gen kernsearch_gaz.cfg
#* ------12x12board-----   
#* for my large computer 
tools/quick-run.sh train kernsearch gaz 50 -conf_str   env_board_size=12:actor_num_simulation=16:kern_search_game_len=144\
:network_type=conv:rand_init_step=5 -gen kernsearch_gaz.cfg
#* 1block for my small computer 
tools/quick-run.sh train kernsearch gaz 50 -conf_str   env_board_size=12:actor_num_simulation=16:kern_search_game_len=144\
:network_type=conv:rand_init_step=5:learner_batch_size=256:learner_training_step=2000:learner_training_display_step=400:nn_num_blocks=1 -gen kernsearch_gaz.cfg
#* 3blocks for my small computer 
tools/quick-run.sh train kernsearch gaz 50 -conf_str   env_board_size=12:actor_num_simulation=16:kern_search_game_len=144\
:network_type=conv:rand_init_step=5:learner_batch_size=128:learner_training_step=4000:learner_training_display_step=800:nn_num_blocks=3 -gen kernsearch_gaz.cfg
#* ------16x16 board ----
#* for my large computer , start with hand crafted kernel   todo maybe we dont need kern_search_len 256  maybe change to 16*(16-5) when hand craft?
tools/quick-run.sh train kernsearch gaz 50 -conf_str \
  env_board_size=16:actor_num_simulation=16:kern_search_game_len=176:network_type=conv:\
rand_init_step=5:min_max_comp=1250,4000 -gen kernsearch_gaz.cfg
tools/quick-run.sh train kernsearch gaz 50 -conf_str \
  env_board_size=16:actor_num_simulation=16:kern_search_game_len=150:network_type=conv:\
rand_init_step=10:min_max_comp=1250,4000 -gen kernsearch_gaz.cfg

#* 16x16 v2  1nn block big sucess   (remember to set hand crafted kernel to false in kernsearch.cpp)
tools/quick-run.sh train kernsearch gaz 50 -conf_str \
  env_board_size=16:actor_num_simulation=16:kern_search_game_len=250:network_type=conv:rand_init_step=7\
   -gen kernsearch_gaz.cfg
#* 16x16 v2  3nn block 
tools/quick-run.sh train kernsearch gaz 50 -conf_str \
  env_board_size=16:actor_num_simulation=16:kern_search_game_len=250:nn_num_blocks=3:network_type=conv:rand_init_step=7\
   -gen kernsearch_gaz.cfg

#*-----3 nn blocks---
#*---start with hand craft kernel------
tools/quick-run.sh train kernsearch gaz 50 -conf_str env_board_size=16:actor_num_simulation=16:\
kern_search_game_len=250:network_type=conv:rand_init_step=5:nn_num_blocks=3 -gen kernsearch_gaz.cfg
#* 
tools/quick-run.sh train kernsearch gaz 50 -conf_str \
  env_board_size=16:actor_num_simulation=16:kern_search_game_len=250:network_type=conv:rand_init_step=7:nn_num_blocks=3\
   -gen kernsearch_gaz.cfg


#* for my small computer , change bs from 1024 to 256, learner_training_step from 500 to 2000  
tools/quick-run.sh train kernsearch gaz 50 -conf_str \
  env_board_size=16:actor_num_simulation=16:kern_search_game_len=250:network_type=conv:rand_init_step=5:learner_batch_size=256:\
learner_training_step=2000:learner_training_display_step=400 -gen kernsearch_gaz.cfg

#* ----------transformer (4x4) board-----------
tools/quick-run.sh train kernsearch gaz 50 -conf_str \
  env_board_size=4:actor_num_simulation=16:kern_search_game_len=20:network_type=transformer:learner_batch_size=128:rand_init_step=0 -gen kernsearch_gaz.cfg
#* network_type=transformer will fail ,  see alphazero_network and test.py to see why, it just takes too long ,  

#**** console kernsearch   
tools/quick-run.sh console kernsearch kernsearch_gaz_1bx256_n16-ed1f10-dirty/  
tools/quick-run.sh console kernsearch kernsearch_gaz_1bx256_n16-19e696-dirty/
tools/quick-run.sh console kernsearch kernsearch_gaz_2bx512_n16-19e696-dirty/
# showboard  
# genmove black
#* gen_multi_move black
# play black 3
# clear_board 

In [ ]:
# todo I think we can feed more parallel cpu,  it is almost sleeping  when doing self play
# todo rotation col_permute stuff
# Actor
actor_num_simulation=50 # simulation number of MCTS
actor_mcts_reward_discount=1 # discount factor for calculating Q values
actor_mcts_value_rescale=false #todo true for games whose rewards are not bounded in [-1, 1], e.g., Atari games
actor_use_random_rotation_features=true #todo true for randomly rotating input features; only supports in alphazero
actor_gumbel_sample_size=16 # hyperparameter for Gumbel Zero; the number of sampled actions
actor_gumbel_sigma_visit_c=50 #param for monotonically increasing transformation sigma in Gumbel Zero
actor_gumbel_sigma_scale_c=1 #param for monotonically increasing transformation sigma in Gumbel Zero
actor_resign_threshold=-0.9 # threshold determining when to resign in the actor
# Zero
zero_num_games_per_iteration=2000 # the nunmber of games to play in each iteration
zero_replay_buffer=20 # replay buffer stores (zero_replay_buffer x zero_num_games_per_iteration) games/sequences
zero_disable_resign_ratio=0.1 #prob to keep playing when winrate is below actor_resign_threshold
# Learner
learner_use_per=false #todo true for enabling Prioritized Experience Replay
learner_training_step=500; learner_batch_size=1024
learner_muzero_unrolling_step=5 # num of steps to unroll for muzero training
learner_n_step_return=0 # num of steps to cal n-step value;usually 0 for board games, 10 for atari games
learner_learning_rate=0.02; learner_num_thread=8 # the number of threads for training  
# Network
nn_num_blocks=1 #  number of the residual blocks
nn_num_hidden_channels=256 # the size of the hidden channels in residual blocks
nn_num_value_hidden_channels=256 # hyperparameter for the model; the size of the hidden channels in the value network

```
debug....  in zeroworker.sh  
change CUDA_VISIBLE_DEVICES=${cuda_devices} ${sp_executable_file} -conf_file ${CONF_FILE} -conf_str "${CONF_STR}" -mode sp 0<&$broker_fd 1>&$broker_fd 
1>&$broker_fd    
to     
2>&$broker_fd   
```

In [ ]:
tools/quick-run.sh train kernsearch gaz 50 -conf_str \
  env_board_size=8:actor_num_simulation=16:kern_search_game_len=64:network_type=conv:actor_mcts_value_rescale=false -gen kernsearch_gaz.cfg

In [ ]:
#lazy version   
python run.py --mode "podman"
python run.py --mode "train" #train go  
python run.py --mode "t"  #train tic tac toe  which will generate tictactoe_gaz

python run.py --mode "tc" #console tic tac tow
#showboard/play black D1/ genmove white
# $ showboard
# $ play black A3
# $ genmove white

tools/quick-run.sh train kernsearchtt gaz 50  #* it is just tictactoc in another env  
tools/quick-run.sh train tictactoe gaz 50
#*console tictactoe                       note that number maychange
tools/quick-run.sh console tictactoe tictactoe_gaz_1bx256_n16-5a959b-dirty/
tools/quick-run.sh console tictactoe tictactoe_gaz_1bx256_n16-19e696-dirty/
$ showboard
$ genmove black
$ genmove white
$ play black A3
$ clear_board
# for valid cmd see minizeo/console/console.cpp  
# ex pv/final_score/clear_board/ list_commands

#* train puzzle2048
tools/quick-run.sh train puzzle2048 gaz 50
tools/quick-run.sh console puzzle2048 puzzle2048_gaz_1bx256_n16-19e696-dirty/
$ showboard  
# note we only have one player
$ genmove black  
# valid action are up down left right
$ play up .      
$ quit


[minizero](https://github.com/rlglab/minizero)

In [ ]:
git clone https://github.com/rlglab/minizero.git
cd minizero # enter the cloned repository  
#* Then, start the runtime environment using the container.
# scripts/start-container.sh #todo
#  must have either podman or docker installed   

#* script is useless this is usefull

podman run --device nvidia.com/gpu=all  --cap-add=SYS_PTRACE --security-opt seccomp=unconfined \
  --network=host --ipc=host --rm -it -v .:/workspace kds285/minizero:latest
+---------------------------------------------------------------------------------------+
root@elton-MS-7D18:/workspace# nvidia-smi
##**** train go
# AlphaZero with 200 simulations
# tools/quick-run.sh train go az 300 -n go_9x9_az_n200 -conf_str env_board_size=9:actor_num_simulation=200

# Gumbel AlphaZero with 16 simulations
tools/quick-run.sh train go gaz 300 -n go_9x9_gaz_n16 -conf_str env_board_size=9:actor_num_simulation=16
#* start training gaz go 1/13 11:23  17:00  100 iter  so roughly 6 hr 100 iter, takes 18 hr to train
#** evaluate go  
# the strength growth for "go_9x9_az_n200"
# tools/quick-run.sh self-eval go go_9x9_az_n200 \
#   -conf_str env_board_size=9:actor_num_simulation=800:actor_select_action_by_count=true:actor_select_action_by_softmax_count=false:actor_use_dirichlet_noise=false:actor_use_gumbel_noise=false
tools/quick-run.sh self-eval go go_9x9_gaz_n16 \
  -conf_str env_board_size=9:actor_num_simulation=800:actor_select_action_by_count=true:actor_select_action_by_softmax_count=false:actor_use_dirichlet_noise=false:actor_use_gumbel_noise=false

### interact with agent   
# play with the "go_9x9_az_n200" model
# tools/quick-run.sh console go go_9x9_az_n200 \
#   -conf_str env_board_size=9:actor_num_simulation=800:actor_select_action_by_count=true:actor_select_action_by_softmax_count=false:actor_use_dirichlet_noise=false:actor_use_gumbel_noise=false

tools/quick-run.sh console go go_9x9_gaz_n16 \
  -conf_str env_board_size=9:actor_num_simulation=800:actor_select_action_by_count=true:actor_select_action_by_softmax_count=false:actor_use_dirichlet_noise=false:actor_use_gumbel_noise=false


ctrl+c
ctrl+d

In [ ]:
# in interact mode  
#https://www.gnu.org/software/gnugo/gnugo_19.html
$ boardsize 7 
$ clear_board   
$ showboard  
$ play black D5  
$ showboard
$ genmove white

#* need to run in terminal (not vscode to show black and white)

In [ ]:
#tictactoe  
#see docs training.md 
#*
tools/quick-run.sh train tictactoe gaz 50  #  takes 15 minutes 
#see docs console.md
#*                                             note that number maychange
tools/quick-run.sh console tictactoe tictactoe_gaz_1bx256_n16-5a959b-dirty/
$ showboard
$ play black A3
$ genmove white

I think 2048 is also nice because its one player   
its in  
minizero/enviroment/stochastic/puzzle2048/

In [ ]:
tools/quick-run.sh train puzzle2048 gaz 50
tools/quick-run.sh console puzzle2048 puzzle2048_gaz_1bx256_n16-aa2b0d-dirty/

# 17:24, 18:51       1hr 20 minute
$ showboard
$ genmove black

to test can we use torch lib in env  
I add $libtorch in enviroment CMakeList.txt  
and also  torch/torch.h in tictactoe.cpp